## Check

In [243]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URLの設定
url = 'http://www1.river.go.jp/cgi-bin/DspWaterData.exe?KIND=2&ID=307111287708010&BGNDATE=20180701&ENDDATE=20241231&KAWABOU=NO'

# HTMLコンテンツの取得
response = requests.get(url)

# BeautifulSoupでHTMLを解析
soup = BeautifulSoup(response.content, 'html.parser')

# pandasでHTMLテーブルを抽出
tables = pd.read_html(str(soup))

# 最初のテーブルを取得（インデックスを0に指定）
df = tables[1]

# 最初の2行をヘッダーとして再設定
# df.columns = df.iloc[1]

# 最初の2行と最後の1行を削除
# df = df.drop([0,  len(df) - 1])
# インデックスをリセット
# df=df.set_index(df.columns[0])


df=df.drop(df.index[[0,1]])
df=df.drop(df.index[len(df)-1])
df=df.set_index(df.columns[0])
# df.index=pd.to_datetime(df.index)
df

/tmp/ipykernel_169051/3648475157.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup))


,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,,,,,,,,,,,,,,,,,,,,,
2018/07/01,0.80,0.80,0.80,0.80,0.80,0.79,0.79,0.78,0.78,0.78,...,0.75,0.75,0.74,0.71,0.70,0.69,0.69,0.68,0.68,0.68
2018/07/02,0.68,0.68,0.67,0.67,0.67,0.67,0.67,0.67,0.66,0.66,...,0.66,0.66,0.66,0.66,0.65,0.65,0.65,0.65,0.65,0.64
2018/07/03,0.64,0.64,0.64,0.64,0.64,0.67,0.72,0.73,0.73,0.71,...,0.64,0.64,0.64,0.64,0.64,0.64,0.64,0.65,0.65,0.65
2018/07/04,0.66,0.68,0.69,0.72,0.74,0.74,0.76,0.74,0.71,0.68,...,0.76,0.77,0.76,0.75,0.74,0.73,0.73,0.72,0.72,0.71
2018/07/05,0.71,0.70,0.70,0.70,0.70,0.70,0.70,0.69,0.71,0.73,...,1.31,1.71,2.59,2.89,3.60,4.39,4.72,4.48,4.16,3.89
2018/07/06,3.63,3.44,3.29,3.16,3.04,2.94,2.80,2.63,2.60,2.57,...,2.62,2.69,2.99,3.97,4.72,5.27,5.77,6.14,6.20,6.20
2018/07/07,5.73,5.58,5.29,5.17,5.07,5.19,5.37,5.60,5.64,5.60,...,4.50,4.24,4.15,3.92,3.69,3.59,3.41,3.28,3.19,3.12
2018/07/08,3.04,2.89,2.83,2.68,2.63,2.52,2.49,2.45,2.41,2.37,...,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測
2018/07/09,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測,...,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測,欠測


---

### arrangeTable(df)
縦軸に日付，横軸に時間があり，その交差に水位が表記されたリストを渡した時，

時刻列と水位列のdataframeを返す．

In [244]:
def arrangeTable(df):
    df=df.drop(df.index[[0,1]])     #最初2行を削除
    df=df.drop(df.index[len(df)-1]) #最後1行を削除
    df=df.set_index(df.columns[0])  #indexをstr(datetime)に変更

    datetimeList=[] #pd.datetime型の日時リスト
    valueList=[] #水位リスト

    #各月毎にループする．
    for index,row in df.iterrows(): 
        for col in df.columns:
            # print(f"Index: {index}, Column: {col}, Value: {value}")

            # 水位リスト作成
            value=row[col]
            valueList.append(value)

            #日時リスト作成
            datetime_Ymd = pd.to_datetime(index, format='%Y/%m/%d', errors='coerce')
            if col==24:
                col=0
                datetime_Ymd = datetime_Ymd + pd.Timedelta(days=1)
            result = datetime_Ymd + pd.to_timedelta(col,unit="h")
            datetimeList.append(result)


    # print(datetimeList)
    # print(valueList)
            
    df_new=pd.DataFrame({
        "dateTime":datetimeList,
        "waterLevel":valueList
    })
    df_new = df_new.set_index('dateTime')

    return df_new

In [245]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# データ格納用リスト
df_list = []

# 開始日と終了日を設定
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2016-3-01')

# 現在の日付を開始日で設定
current_date = start_date

while current_date <= end_date:
    # URLの構築
    url = f'http://www1.river.go.jp/cgi-bin/DspWaterData.exe?KIND=2&ID=307111287708010&BGNDATE={current_date.strftime("%Y%m%d")}&ENDDATE=20241231&KAWABOU=NO'

    # HTMLコンテンツの取得
    response = requests.get(url)

    # BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(response.content, 'html.parser')

    # pandasでHTMLテーブルを抽出
    tables = pd.read_html(str(soup))

    # 最初のテーブルを取得（インデックスを1に指定）
    df = tables[1]
    # print(df)
    # テーブル整理
    df = arrangeTable(df)
    # print(df)
    # リストに追加
    df_list.append(df)

    # 次の月に進む
    current_date += pd.DateOffset(months=1)

# 最後に、データフレームリストを結合
final_df = pd.concat(df_list, ignore_index=False)

# 結果の表示（任意）
print(final_df)

/tmp/ipykernel_169051/3252907902.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup))
/tmp/ipykernel_169051/3252907902.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup))


                    waterLevel
dateTime                      
2016-01-01 01:00:00       0.55
2016-01-01 02:00:00       0.55
2016-01-01 03:00:00       0.55
2016-01-01 04:00:00       0.55
2016-01-01 05:00:00       0.55
...                        ...
2016-03-31 20:00:00       0.52
2016-03-31 21:00:00       0.52
2016-03-31 22:00:00       0.52
2016-03-31 23:00:00       0.52
2016-04-01 00:00:00       0.52

[2184 rows x 1 columns]


/tmp/ipykernel_169051/3252907902.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup))
